In [ ]:
# --- MODULOS PROPIOS

from model_functions import time_discretization, len_trip, distance, estimate_time, cost_fuel
from compatibility_functions import compatibility, comp_T
#from data_paper import data_paper
from graph_construction import create_nodes, create_archs
from plot_functions import plot_city, plot_graph
from generate_data import generate_data
from initial_model import initial_model
from lp_functions import from_V_to_TDS, calculate_cost, calculate_fuel
from export_data import export_data
import random

In [ ]:
seed = 887335#random.randint(0, 1e6)#            # Semilla para generar una instancia aleatoria fija.

K = 0.1                                 # Constante para transformar el consumo de combustible en tiempo recargando.

num_travels = 100                        # Número de viajes.
num_depots = 3                          # Número de depósitos.
num_charge_stations = 5                 # Número de estaciones de carga.
num_passenger_stations = 10             # Número de estaciones de pasajeros.

w = 100.0                               # Capacidad de combustible de cada vehículo.

speed_bus = 3.0                         # Velocidad del bus (km/h).
fuel_per_distance = 2.0                 # Combustibple requerido por distancia ej. litros de gasolina/km.
cost_per_distance = 1.0                 # Costo por distancia (sucres/km).

h_s = 6.0                               # Hora de inicio de la jornada.
h_f = 20.0                              # Hora de final de la jornada.

time_divisions = 50                     # Número de divisiones temporales para discretización.

v_max = 100                             # Capacidad máxima de un depósito dado (más adelante, la capacidad se genera aleatoriamente entre v_min y v_max).
v_min = 10                              # capacidad mínima de un depósito dato


size_square = 10                        # Lado de una ciudad cuadrada (km).


# Se considera la capacidad limitada de las estaciones de carga.
max_capacity_charge_station = 200         # Capacidad máxima de una estacion de carga por unidad de tiempo.
                                        # (mas adelante, la capacidad se genera aleatoriamente entre min_capacity_charge_station y max_capacity_charge_station).

min_capacity_charge_station = 30       # Capacidad mínima de una estación de carga en un tiempo dado  

In [ ]:
# Llamando a las funciones en tuplas para mejorar la legibilidad.
(time_window,
time_intervals, 
time_dic, 
list_time,
delta)              = time_discretization(h_s, h_f, time_divisions)


In [ ]:
#Se genera datos para una instancia.
(Travels, 
 Depots, 
 Stations_chrg, 
 Stations_chrg_time, 
 coord_passenger_stations, 
 coord_depots, 
 coord_charge_stations, 
 T_passenger_stations, 
 T_ab, 
 D_v,
 rs_kt)                 = generate_data(seed, num_travels, num_depots, num_charge_stations, num_passenger_stations, size_square, v_max, v_min, time_window, max_capacity_charge_station, min_capacity_charge_station, list_time, speed_bus)




# Se calcula el tiempo entre estaciones viajes, depósitos y estaciones de carga.
t                       = estimate_time(Travels, Depots, Stations_chrg, coord_passenger_stations, T_passenger_stations, coord_depots, coord_charge_stations, speed_bus)

(H_list,
 H_list_inv,
h, 
h_inv,
V,
V_visual,
Stations_chrg_time_dic,
Stations_chrg_time_dic_inv)= create_nodes(Travels, Depots, Stations_chrg_time, num_travels, num_depots,num_charge_stations)


#Se calcula las compatibilidades.
(dic_comp, 
 dic_comp_F,
 list_comp_FT)            = compatibility(Travels, Depots, Stations_chrg, Stations_chrg_time, T_passenger_stations, T_ab, t, time_dic)


Hk = {k:[h[h_node] for h_node in H_list if h_node[0] in Travels+[k] ] for k in Depots}

(A1, 
 A2, 
 A3, 
 A4, 
 A5, 
 A6, 
 A7,
 A, 
 A_list,
 delta_mas,
 delta_menos,
 Ak,
 cost,
 fuel)                = create_archs(Travels, Depots, h, H_list, dic_comp, dic_comp_F, T_ab, time_dic, t, Hk, fuel_per_distance, T_passenger_stations, coord_passenger_stations, Stations_chrg, coord_depots, coord_charge_stations, cost_per_distance )
#Se estiman los costos y el combustible.
(cost_old,
fuel_old)                   = cost_fuel(Travels, Depots, Stations_chrg, coord_passenger_stations, T_passenger_stations, T_ab, coord_depots, coord_charge_stations, h_inv, A, A1, A2, A3, A4, A5, A6, A7, cost_per_distance, fuel_per_distance)


In [ ]:
keys_mas = cost.keys()
keys_old = cost_old.keys()

asd_count = 0
for asd in keys_old:
    if asd not in keys_mas:
        asd_count += 1
        print(asd)
        
print(asd_count)
print(len(keys_old))
print(len(keys_mas))

In [ ]:
list(fuel.values())==list(fuel_old.values())

In [ ]:
# Grafica la ciudad
#plot_city(T_passenger_stations, T_ab, coord_passenger_stations, coord_depots, coord_charge_stations, size_square)
'''
print(f'Viaje\tstation\thoras')
for i in Travels:
    print(f'{i}\t{T_passenger_stations[i]}\t{T_ab[i]}')
'''

# Generación de Columnas

 Sea $\Omega$ el conjunto de todos los posibles itinerarios para una instancia del problema. Para un itinerario en particular $p\in\Omega$ sea el valor $a_{ip}$ una contstante binaria que toma el valor de 1 si y solo si la ruta $p$ incluye el viaje $T_i$, y $b_p^k$ una constante binaria que toma el valor de uno si y solo si la ruta $p$ comineza y termina en $D_k$. 
 
 
 
 
 
 Entonces, el problema se puede formular como:


\begin{align}
  \min \quad & \sum_{p\in \Omega} q_p x_p  \\
  \text{sujeto a}\nonumber\\
  \quad & \sum_{p\in \Omega}a_p^i x_p = 1,        &   i=1,\cdots,n, \\
        & \sum_{p\in \Omega}b_p^j x_p \leq r_j,   &   j=1,\cdots,d,  \\
        & \sum_{p\in \Omega}c_p^{kt} x_p \leq r_{kt},   &   k=1,\cdots,s, t\in \mathscr{T} \\
        & x_p\in \{0,1\}, \forall p \in \Omega. \nonumber
\end{align}

Nos referimos a este problema lineal como el problema máster. Cuando el problema se resuelve usando un subconjunto $\Omega'\subseteq\Omega$ de todos los posibles itinerarios, nos referimos a este como el problema máster restringido. 

Empezamos resolviendo el dual de la relajación lineal para un conjunto pequeño de posibles itinerarios candidatos y luego usamos el dual de la solución para añadir nuevos itinerarios para añadir al problema máster restringido.


Para el esquema de generación de columnas se considera que el origen y destino es $\theta_j$, los vértices en el conjunto $\hat{H}$ son nodos de recarga, la capacidad es $w$, y para un arco $(v_1,v_2)\in E$, los valores de combustible son $f''(v_1,v_2) = f'(v_1,v_2)$ y los valores de costo son:



$$
c''(v_1,v_2)= \begin{cases}
c'(v_1,v_2)-\pi_i   &\text{para }v_1\in T, v_2 \in \hat{V}\\
c'(v_1,v_2)+\xi_{kt}     &\text{para }v_1\in \hat{H}, v_2 \in \hat{V}, v_1 = h(z_1,\sigma_{k_1,t_1}) \text{ si } k_1=k \text{ y } t_1 =t\\
c'(v_1,v_2)+\rho_j  &\text{para }v_1\in D, v_2 \in \hat{V}.
\end{cases}
$$





In [ ]:
def which_node_is(node):
    if node in Travels:
        print("Travels")
    elif node in Depots:
        print("Depots")
    elif node in Stations_chrg:
        print("Stations_chrg")
    elif node in list(Stations_chrg_time_dic.keys()):
        print(f"Stations_chrg_time\t{Stations_chrg_time_dic[node]}")
    elif node in H_list_inv:
        print(f"h\t{h_inv[node]}")


In [ ]:
#export_data(time_dic, w, seed, V, T_ab, D_v,h_inv, rs_kt, cost, fuel, A1, A2, A3, A4, A5, A6, A7, single_file = True)

In [ ]:
#from import_data import import_data

#import_data()

In [ ]:
num_max_it          = 1900        #número de máximas iteraciones para el modelo (para no saturar gurobi ni el PC).

(new_paths, 
 dic_cost_path, 
 x_op,
 z_op)              = initial_model(seed,list_comp_FT,K, Travels, Depots, Stations_chrg_time, Stations_chrg_time_dic, Stations_chrg_time_dic_inv, D_v, rs_kt, h, Stations_chrg, dic_comp, cost, list_time, w, fuel, h_inv, dic_comp_F, T_ab, num_max_it, delta, time_window, t, time_dic)

 
 # Se imprime la solución.
for i in x_op:
    if x_op[i] > 0:
        path_TDS = from_V_to_TDS(new_paths[i], Travels, Depots, Stations_chrg, h_inv, visible=True)    
        coste = calculate_cost(new_paths[i], cost)
        print(path_TDS, coste)
 



In [ ]:
# Se visualiza el grafo.
#plot_graph(V, V_visual, A_list, cost, fuel)

# MODELO DE COMPROBACION
---------------

A partir de aqui empieza otro


Se definen los conjuntos:

$$
H(s,t):=\{ h(z,\sigma_{st}) | z \in  T \cup D \}
$$

$$
H^T(z) := \{h(z,\sigma_{st}) \in H \}
$$


Se calculan los valores
- $M_1 = 2 \sum_{(i,j)\in A}f_{ij},$
- $M_2 = 2t_{max} + Kw + b_{max},$
- $t_{max} := \max\{ t(i,j) | (i,j) \in A \},$
- $b_{max} := \{b_i | i \in T \},$


In [ ]:
# M1
fuel_accumulated = 0 
for (i,j) in A:
    fuel_accumulated += fuel[(i,j)]
M1 = 2*fuel_accumulated

# M2
tmax = max(list(t.values()))
bmax = max([ ab[1] for ab in list(T_ab.values())])
M2 = 2*tmax + K*w + bmax

In [ ]:
# para hacer un multigrafo, se debe dividir el conjunto de H porque no debe contener todos los depósitos para una copia k del grafo
#Hk = {k:[h[h_node] for h_node in H_list if h_node[0] in Travels+[k] ] for k in Depots}

#Ak_old = [(i, j, k) for (i,j) in A for k in Depots if i in Travels+[k]+Hk[k] and j in Travels+[k]+Hk[k]]

#delta_mas_old= { (i,k) : [ j1 for (i1,j1,k1) in Ak if i1 == i and k == k1] for i in V for k in Depots}
#delta_menos_old= { (j,k) : [ i1 for (i1,j1,k1) in Ak if j1 == j and k == k1 ] for j in V for k in Depots}


In [ ]:
list(fuel_old.values()) == list(fuel.values())

In [ ]:
Depots

In [ ]:
Hk_complete = {k:[h_arch for h_arch in H_list if h_arch[0] in Travels+[k] ] for k in Depots}

In [ ]:
H_stk = { (s1,t1,k1): [h[key] for key in Hk_complete[k1] if key[1] == (s1,t1)] for (s1,t1) in Stations_chrg_time for k1 in Depots}
H_T = { i:[h[index] for index in H_list if index[0] == i  ] for i in Travels}

In [ ]:
import gurobipy as gp

# Crear el objeto del modelo
model2 = gp.Model()

# Definir las variables de decisión
x = model2.addVars(Ak, vtype=gp.GRB.BINARY, name='x')
F = model2.addVars(V, lb = 0, ub = w, vtype=gp.GRB.CONTINUOUS, name='F')


# Definir la función objetivo 
model2.setObjective(gp.quicksum(cost[(arco[0],arco[1])]*x[arco] for arco in Ak ), sense=gp.GRB.MINIMIZE)


# Restricciones

# Primera restricción: se deben satisfacer todos los viajes
model2.addConstrs((gp.quicksum(x[(i,j,k)] for k in Depots for j in delta_mas[i,k] ) == 1 for i in Travels), name = '1ra_restr')

# Segunda restricción: se respeta la capacidad del depósito
model2.addConstrs((gp.quicksum(x[(k,j,k)] for j in delta_mas[k,k]) <= D_v[k] for k in Depots), name = '2da_restr')
    
# Tercera restricción: conservación de flujo
model2.addConstrs((gp.quicksum(x[(j,i,k)] for j in delta_menos[i,k]) - gp.quicksum(x[(i,j,k)] for j in delta_mas[i,k]) == 0 for k in Depots for i in Travels+Hk[k]+[k]), name = '3ra_restr')

# Cuarta restricción: capacidad de las estaciones de carga
#model2.addConstrs((gp.quicksum(x[(h1, j, k)] for h1 in H_st[(s1, t1)] for k in Depots for j in delta_mas[h1,k] ) <= rs_kt[(s1, t1)] for s1 in Stations_chrg for t1 in list_time), name='4ta_restr')
model2.addConstrs((gp.quicksum(x[(h1, j, k)] for k in Depots for h1 in H_stk[(s1, t1, k)] for j in delta_mas[h1,k]  ) <= rs_kt[(s1, t1)] for s1 in Stations_chrg for t1 in list_time), name='4ta_restr')

# Quinta restricción: 
model2.addConstrs((F[j] >= F[i] + fuel[(i,j)] - (1-x[(i,j,k)])*M1 for (i,j,k) in Ak if i not in Depots + H_list_inv), name = '5ta_restr')

# Sexta restricción: 
model2.addConstrs((F[j] >= fuel[(i,j)] - (1-x[(i,j,k)])*M1 for (i,j,k) in Ak if i in Depots + H_list_inv), name = '6ta_restr')


# Séptima restricción 
for (h1, j) in A:
    if h1 in H_list_inv and j in Travels :
            model2.addConstr( time_window[0] + delta*(h_inv[h1][1][1] + 1) + K*F[h1] + t[(h_inv[h1][1][0], j)] 
                             <= T_ab[j][0] + (1 - gp.quicksum(x[(h1,j,k2)] for k2 in Depots if (h1,j,k2) in Ak ))*M2 , name=f'7ma_restr_{j}_{h1}' )


model2.Params.OutputFlag = 0
model2.Params.LogFile = "salida_modelo_arcos.log"
model2.optimize()


optimal_value = "Null"
if model2.status == gp.GRB.OPTIMAL:
    # Obtener el valor óptimo de la función objetivo
    optimal_value = model2.objVal
    print("-"*100)
    print('Valor óptimo de la función objetivo:', optimal_value)


model2.write("modelo_verificacion.lp")


In [ ]:
print(H_list[0])
#print(h[(1, (34, 0))])
print(Depots)

In [ ]:
#h[(1, (34, 0))] in Hk[31]

In [ ]:
# Verificar si el modelo es infactible
if model2.status == gp.GRB.INFEASIBLE:
    print('El modelo es infactible.')

    # Calcular el IIS
    model2.computeIIS()

    # Imprimir el IIS
    print('El IIS es:')
    for c in model2.getConstrs():
        if c.IISConstr:
            print('%s' % c.constrName)


In [ ]:
archs_new_model = []

if model2.status != gp.GRB.INFEASIBLE:
    for i in Ak:
        if x[i].x > 0:
            archs_new_model.append([i[0],i[1],i[2]])
            

In [ ]:

dic_caminos = {}
path_id = 0

contador = 0

if model2.status != gp.GRB.INFEASIBLE:
    # código para juntar los arcos solución en sus respectivos caminos
    for arch in archs_new_model:
        if arch[0] in Depots:
            initial_depot = arch[0]
            dic_caminos[path_id] = [initial_depot]
    
            next_node = arch[1]
            dic_caminos[path_id].append(next_node)

            while next_node != initial_depot:
                for arch1 in archs_new_model:
                    if arch1[0] == next_node:
                                                               
                        next_node = arch1[1]
                        dic_caminos[path_id].append(next_node) 

                        contador += 1
                    if next_node == initial_depot:
                        break

            path_id += 1

    

In [ ]:
#plot_graph(V, V_visual, A_list, cost, fuel)

In [ ]:
def viewer_time(camine,fuele):
    index_fuel = 0
    for index, node in enumerate(camine):
        if node in H_list_inv:

            print(f'------ Para el h-nodo {h_inv[node]} ------')

            # la componente temporal del nodo h es: h_inv[node][1][1]
            if camine[index - 1] in Travels:
                print("\t\tANTES")
                print(f'et(T_{index - 1}) + t(T_{index - 1}, S_{h_inv[node][1][0]}) = {T_ab[index - 1][1]} + {t[index-1,h_inv[node][1][0]]} = {T_ab[index - 1][1] + t[index - 1,h_inv[node][1][0]]}')
                print(f't = {h_inv[node][1][1]} corresponde a: \t{time_dic[h_inv[node][1][1]]}')

            if camine[index - 1] in Depots:
                print("\t\tANTES")
                print("viene del depósito")
            
            if camine[index + 1] in Depots:
                print("\t\tDESPUÉS")
                print("llega al depósito")

            if camine[index + 1] in Travels:
                print("\t\tDESPUÉS")
                print(f'et({h_inv[node][1]}) + K*F_({h_inv[node]}) + t(S_{h_inv[node][1][0]}, T_{index + 1}) = {time_dic[h_inv[node][1][1]][1]} + {K*fuele[index_fuel]} + {t[h_inv[node][1][0], index + 1]} = {time_dic[h_inv[node][1][1]][1] + K*fuele[index_fuel] + t[h_inv[node][1][0], index + 1]} ')
                print(f'bt(T_{index + 1}) = {T_ab[index+1][1]}')
                index_fuel += 1
    print('---- FIN DE ESTE CAMINO ----')
    print('----'*10)



In [ ]:
# Se imprime la solución.

print("\t\tmodelo generación de columnas")

if z_op < 10000:
    print('cost\tfuel\t\t\tpath_TDS')
    for i in x_op:
        if x_op[i] > 0:
            path_TDS = from_V_to_TDS(new_paths[i], Travels, Depots, Stations_chrg, h_inv, visible=True)  
            coste = calculate_cost(new_paths[i], cost)
            fuele = calculate_fuel(new_paths[i], fuel, H_list_inv)
            print(f'{coste}\t{fuele}\t\t\t{path_TDS}')
            #viewer_time(new_paths[i],fuele)

else:
    print("------------ NO VALIO ------------")
    for i in x_op:
        if x_op[i] > 0:
            path_TDS = from_V_to_TDS(new_paths[i], Travels, Depots, Stations_chrg, h_inv, visible=True)    
            coste = calculate_cost(new_paths[i], cost)
            print(path_TDS, coste)

print("\n")
print("\t\tmodelo comprobación")
print('cost\tfuel\t\t\tpath_TDS')


for i in dic_caminos:
    path_TDS = from_V_to_TDS(dic_caminos[i], Travels, Depots, Stations_chrg, h_inv, visible=True)    
    coste = calculate_cost(dic_caminos[i], cost)
    fuele = calculate_fuel(dic_caminos[i], fuel, H_list_inv)

    print(f'{coste}\t{fuele}\t\t\t{path_TDS}')

    #viewer_time(dic_caminos[i],fuele)

print("\n")
print("\t\tmodelo\tmodelo comprobación")
print(f"z_optimo\t{z_op}\t{optimal_value}")

print(f'\n\nla semilla es {seed}')